#Notebook for testing performance of NLC classification
[Watson Developer Cloud](https://www.ibm.com/watsondevelopercloud) is a platform of cognitive services that leverage machine learning techniques to help partners and clients solve a variety business problems. Furthermore, several of the WDC services fall under the **supervised learning** suite of machine learning algorithms, that is, algorithms that learn by example. This begs the questions: "How many examples should we provide?" and "When is my solution ready for prime time?"

It is critical to understand that training a machine learning solution is an iterative process where it is important to continually improve the solution by providing new examples and measuring the performance of the trained solution. In this notebook, we show how you can compute important Machine Learning metrics (accuracy, precision, recall, confusion_matrix) to judge the performance of your solution. For more details on these various metrics, please consult the **[Is Your Chatbot Ready for Prime-Time?](https://developer.ibm.com/dwblog/2016/chatbot-cognitive-performance-metrics-accuracy-precision-recall-confusion-matrix/)** blog.


<br> The notebook assumes you have already created a Watson [Natural Language Classifier](https://www.ibm.com/watson/developercloud/nl-classifier.html) instance and trained a classifier. </br>
<br> To leverage this notebook, you need to provide the following information</br>
* Credentials for your NLC instance (username and password)
* id for your trained classifier (this is returned when you train an NLC classifier)
* csv file with your text utterances and corresponding class labels
* results csv file to write the results to

Note that the input test csv file should have a header with the fields **text** and **class**. 

In [ ]:
#Import utilities
import json
import csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas_ml
from pandas_ml import ConfusionMatrix
from watson_developer_cloud import NaturalLanguageClassifierV1

Provide the path to the parms file which includes credentials to access your NLC service as well as the input
test csv file and the output csv file to write the output results to.

In [ ]:
# Provide complete path to the file which includes all required parms
# A sample parms file is included (example_parms.json)
nlcParmsFile = 'PATH to your PARMS file'
parms = ''
with open(nlcParmsFile) as parmFile:
    parms = json.load(parmFile)

url=parms['url']
user=parms['user']
password=parms['password']
nlc_id=parms['nlc_id']
test_csv_file=parms['test_csv_file']
results_csv_file=parms['results_csv_file']
confmatrix_csv_file=parms['confmatrix_csv_file']

json.dumps(parms)

# Create an object for your NLC instance
natural_language_classifier = NaturalLanguageClassifierV1(
  username=user,
  password=password)

Define useful methods to classify using trained NLC classifier.

In [ ]:
# Given a text string and a pointer to NLC instance and classifierID, get back NLC response
def getNLCresponse(nlc_instance,classifierID,string):
    classes = nlc_instance.classify(classifierID, string)
    return classes

# Process multiple text utterances (provided via csv file) in batch. Effectively, read the csv file and for each text
# utterance, get NLC response. Aggregate and return results.
def batchNLC(nlc_instance,classifierID,csvfile):
    test_classes=[]
    nlcpredict_classes=[]
    nlcpredict_confidence=[]
    text=[]
    i=0
    with open(csvfile, 'r') as csvfile:
        csvReader=csv.DictReader(csvfile)
        for row in csvReader:
            test_classes.append(row['class'])
            nlc_response = getNLCresponse(nlc_instance,classifierID,row['text'])
            nlcpredict_classes.append(nlc_response['classes'][0]['class_name'])
            nlcpredict_confidence.append(nlc_response['classes'][0]['confidence'])
            text.append(row['text'])
            i = i+1
            if(i%250 == 0):
                print ('Processed ', i, ' records')
        print ('Finished processing ', i, ' records')        
    return test_classes, nlcpredict_classes, nlcpredict_confidence, text

# Plot confusion matrix as an image
def plot_conf_matrix(conf_matrix):
    plt.figure()
    plt.imshow(conf_matrix)
    plt.show()

# Print confusion matrix to a csv file
def confmatrix2csv(conf_matrix,labels,csvfile):
    with open(csvfile, 'w') as csvfile:
        csvWriter = csv.writer(csvfile)
        row=list(labels)
        row.insert(0,"")
        csvWriter.writerow(row)
        for i in range(conf_matrix.shape[0]):
            row=list(conf_matrix[i])
            row.insert(0,labels[i])
            csvWriter.writerow(row)
            

In [ ]:
# This is an optional step to quickly test response from NLC for a given utterance
#testQ='can I reset my password'
#results = getNLCresponse(natural_language_classifier,nlc_id,testQ)
#print(json.dumps(results, indent=2))

Call NLC on the specified csv file and collect results.

In [ ]:
test_classes,nlcpredict_classes,nlcpredict_conf,text=batchNLC(natural_language_classifier,nlc_id,test_csv_file)

In [ ]:
# print results to csv file including original text, the correct label, 
# the predicted label and the confidence reported by NLC.
csvfileOut=results_csv_file
with open(csvfileOut, 'w') as csvOut:
    outrow=['text','true class','NLC Predicted class','Confidence']
    csvWriter = csv.writer(csvOut,dialect='excel')
    csvWriter.writerow(outrow)
    for i in range(len(text)):
        outrow=[text[i],test_classes[i],nlcpredict_classes[i],str(nlcpredict_conf[i])]
        csvWriter.writerow(outrow)

In [ ]:
# Compute confusion matrix
labels=list(set(test_classes))
nlc_confusion_matrix = confusion_matrix(test_classes, nlcpredict_classes, labels)
nlcConfMatrix = ConfusionMatrix(test_classes, nlcpredict_classes)

In [ ]:
# Print out confusion matrix with labels to csv file
confmatrix2csv(nlc_confusion_matrix,labels,confmatrix_csv_file)

In [ ]:
%matplotlib inline
nlcConfMatrix.plot()

In [ ]:
# Compute accuracy of classification
acc=accuracy_score(test_classes, nlcpredict_classes)
print ('Classification Accuracy: ', acc)

In [ ]:
# print precision, recall and f1-scores for the different classes
print(classification_report(test_classes, nlcpredict_classes, labels=labels))

In [ ]:
#Optional if you would like each of these metrics separately
#[precision,recall,fscore,support]=precision_recall_fscore_support(test_classes, nlcpredict_classes, labels=labels)
#print ('precision: ', precision)
#print ('recall: ', recall)
#print ('f1 score: ', fscore)
#print ('support: ', support)